In [11]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
#from BCEmbedding.tools.langchain import BCERerank
from FlagEmbedding import FlagReranker
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain.retrievers import ContextualCompressionRetriever
import csv
import os
import time
import pickle
from zhipuai import ZhipuAI
import csv
import json

In [12]:
model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章:"
)

In [13]:
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up 
db = FAISS.load_local('../data/embedding_2048_512',embedding,allow_dangerous_deserialization = True)
retriever=db.as_retriever( search_kwargs={"k":9})

In [6]:

prompt = hub.pull("rlm/rag-prompt")
print(f"prompt is: {prompt}")
print("--------")
llm = ChatOpenAI(
    model_name="GLM-4-PLus",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4",
    openai_api_key="3138eb04ab176ee38855bc7bd5883868.RZfclCazuZkKXA1f",
    streaming=False,
)

/home/vipuser/miniconda3/envs/vllm/lib/python3.11/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


prompt is: input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]
--------


In [7]:
def format_docs(docs):
    return "".join(doc.page_content for doc in docs)
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt 
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [10]:

extracted_data = []

with open('../data/val.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        question = data['input_field']
        prompt="以下问题只关于TuGraph-DB\n"+ question +"\n要求：直接回答结果，如果不知道回答不知道，不要添加额外内容，不要进行推测。回答长度不要超过100字。"#提示词这块是需要工程化来做的
        result = rag_chain_with_source.invoke(question)
        extracted_info = {"id": data["id"],"output_field": result['answer']}
        extracted_data.append(extracted_info)
        print(question,result['answer'])
        print('\n')
#print(extracted_data)
# 将提取的数据写入到answer.jsonl文件中
with open('../result/val_ans.jsonl', 'w',encoding='utf-8') as f:
    for item in extracted_data:
        # 将每个字典转换为JSON格式的字符串，并写入文件
        json_line = json.dumps(item,ensure_ascii=False) + '\n'
        f.write(json_line)
print("数据已写入到val_ans.jsonl文件中。")

RPC 及 HA 服务中，verbose 参数的设置有几个级别？
数据已写入到val_ans.jsonl文件中。


In [ ]:
with open('data/test1.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        question = data['input_field']
        prompt="以下问题只关于TuGraph-DB\n"+ question +"\n要求：直接回答结果，如果不知道回答不知道，不要添加额外内容，不要进行推测。回答长度不要超过100字。"#提示词这块是需要工程化来做的
        result = rag_chain_with_source.invoke(question)
        extracted_info = {"id": data["id"],"output_field": result['answer']}
        extracted_data.append(extracted_info)
        print(question,result['answer'])
        print('\n')
#print(extracted_data)
# 将提取的数据写入到answer.jsonl文件中
with open('result/answer2.jsonl', 'w',encoding='utf-8') as f:
    for item in extracted_data:
        # 将每个字典转换为JSON格式的字符串，并写入文件
        json_line = json.dumps(item,ensure_ascii=False) + '\n'
        f.write(json_line)
print("数据已写入到answer.jsonl文件中。")